# **Bayut Oman Rentals**

### **Installing Selenium Library**

### **Setting The Environment**

In [71]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests

### **Opening the Bayut rental page using Chrome**

In [74]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

base_url = 'https://www.bayut.om/en/oman/properties-for-rent/page/{}/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

properties = {
    "Title": [],
    "Location": [],
    "Price": [],
    "Size": [],
    "Bedrooms": [],
    "Listing_Type": [],
    "Link": []
}

max_pages = 55

for page_num in range(1, max_pages + 1):
    url = base_url.format(page_num)
    print(f"Scraping page {page_num}...")

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        ads = soup.find_all('article', class_='fbc619bc')

        if not ads:
            print("No more ads found.")
            break

        for ad in ads:
            # Initialize placeholders
            title = None
            link = None
            location = None
            price = None
            size = None
            bedrooms = None
            listing_type = "For Rent"

            # Title and Link
            title_tag = ad.find('a', attrs={'aria-label': 'Listing link'})
            if title_tag:
                title = title_tag.get('title')
                href = title_tag.get('href')
                link = 'https://www.bayut.om' + href if href else None

            # Location
            location_tag = ad.find('div', attrs={'aria-label': 'Location'})
            if location_tag:
                location = location_tag.text.strip()
            elif title:
                match = re.search(r'in\s(.+)', title)
                if match:
                    location = match.group(1)

            # Price
            price_tag = ad.find('span', attrs={'aria-label': 'Price'})
            if price_tag:
                price = price_tag.text.strip()

            # Fetch detail page to get Bedrooms and Size
            if link:
                try:
                    detail_response = requests.get(link, headers=headers)
                    detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

                    # Bedrooms
                    bed_container = detail_soup.find('span', attrs={'aria-label': 'Beds'})
                    if bed_container:
                        bed_text = bed_container.find('span', class_=re.compile(r'_140e6903'))
                        if bed_text:
                            bedrooms = bed_text.text.strip()

                    # Size
                    size_container = detail_soup.find('span', attrs={'aria-label': 'Area'})
                    if size_container:
                        size_text = size_container.find('span', class_=re.compile(r'_140e6903'))
                        if size_text:
                            size_inner = size_text.find('span')
                            if size_inner:
                                size = size_inner.text.strip()

                except Exception as e:
                    print(f"Error fetching detail from {link}: {e}")

            # Append to dictionary
            properties["Title"].append(title)
            properties["Link"].append(link)
            properties["Location"].append(location)
            properties["Price"].append(price)
            properties["Size"].append(size)
            properties["Bedrooms"].append(bedrooms)
            properties["Listing_Type"].append(listing_type)

        time.sleep(1)  # Be polite to the server

    except Exception as e:
        print(f"Error scraping page {page_num}: {e}")

# Final check for length consistency (optional)
lengths = {k: len(v) for k, v in properties.items()}
if len(set(lengths.values())) > 1:
    print("Warning: Column lengths are not equal!")
    print(lengths)


Scraping page 1...
Error scraping page 1: HTTPSConnectionPool(host='www.bayut.om', port=443): Read timed out. (read timeout=None)
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Error fetching detail from https://www.bayut.om/en/property/details-130250150.html: HTTPSConnectionPool(host='www.bayut.om', port=443): Read timed out. (read timeout=None)
Error fetching detail from https://www.bayut.om/en/property/details-129421978.html: HTTPSConnectionPool(host='www.bayut.om', port=443): Read timed out. (read timeout=None)
Scraping page 8...
Error scraping page 8: HTTPSConnectionPool(host='www.bayut.om', port=443): Read timed out. (read timeout=None)
Scraping page 9...
Error scraping page 9: HTTPSConnectionPool(host='www.bayut.om', port=443): Read timed out. (read timeout=None)
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping pag

In [75]:
# Save to CSV
df = pd.DataFrame(properties)
df.to_csv("bayut_properties_for_rent.csv", index=False)
print(f"\nScraping complete. Saved {len(df)} listings to 'bayut_properties_for_rent.csv'")


Scraping complete. Saved 1248 listings to 'bayut_properties_for_rent.csv'
